In [45]:
# Dependencies
from bs4 import BeautifulSoup
import requests
import os
import pymongo

In [46]:
url = 'https://mars.nasa.gov/news/'
headers = {'User-Agent': 'Mozilla/5.0'}
html = requests.get(url, headers=headers).text
# print(html)

In [47]:
# init pymongo to work with mongodb
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

# define db and collection for part i
db = client.scrape_mars_db
collection = db.parti

# Create BeautifulSoup object; parse with 'html.parser'
soup = BeautifulSoup(html, 'html.parser')

In [48]:
# soup

In [49]:
results = soup.find_all('div', class_='slide')
# results

In [50]:
# loop through returned results
for result in results:
    
    # retrieve title and summary
    title = result.find('div', class_='content_title')
    summary = result.find('div',class_='rollover_description_inner')
    
    # access content
    news_title = title.a.text
    news_p = summary.text
    print('[Article Title]')
    print(news_title)
    print('[Summary]')
    print(news_p)
    print('-------')
    # add document to parti collection in mongodb
    post = {
        'title': news_title,
        'summary': news_p
    }
    collection.insert_one(post)

[Article Title]

NASA Prepares for Moon and Mars With New Addition to Its Deep Space Network

[Summary]

Robotic spacecraft will be able to communicate with the dish using radio waves and lasers.

-------
[Article Title]

NASA Administrator Statement on Moon to Mars Initiative, FY 2021 Budget

[Summary]

Jim Bridenstine addresses NASA's ambitious plans for the coming years, including Mars Sample Return.

-------
[Article Title]

NASA's Mars 2020 Rover Closer to Getting Its Name

[Summary]

155 students from across the U.S. have been chosen as semifinalists in NASA's essay contest to name the Mars 2020 rover, and see it launch from Cape Canaveral this July.

-------
[Article Title]

NASA Invites Students to Name Mars 2020 Rover

[Summary]

Through Nov. 1, K-12 students in the U.S. are encouraged to enter an essay contest to name NASA's next Mars rover.

-------
[Article Title]

NASA's Curiosity Mars Rover Finds a Clay Cache

[Summary]

The rover recently drilled two samples, and both sh

In [7]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import time

In [8]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [9]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

In [10]:
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [11]:
html = browser.html
soup = bs(html, 'html.parser')
# soup

# define db and collection for part ii
collection = db.partii

In [12]:
html = browser.html
soup = bs(html, 'html.parser')
browser.links.find_by_partial_text('FULL IMAGE')
image = soup.find('a', class_='button fancybox')
image_url = image['data-fancybox-href']
firstparturl = 'https://www.jpl.nasa.gov/'
featured_image_url = firstparturl + image_url
feat_img = {
    'url':featured_image_url
}
# print(featured_image_url)
collection.insert_one(feat_img)


In [13]:
import pandas as pd

In [14]:
url = 'https://space-facts.com/mars/'

In [15]:
tables = pd.read_html(url)
tables

[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers,
   Mars - Earth Comparison             Mars            Earth
 0               Diameter:         6,779 km        12,742 km
 1                   Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 2                  Moons:                2                1
 3      Distance from Sun:   227,943,824 km   149,598,262 km
 4         Length of Year:   687 Earth days      365.24 days
 5            Temperature:    -153 to 20 °C      -88 to 58°C,
           

In [16]:
df = tables[0]
# df_2 = tables[1]
# df_3 = tables[2]
df_col_rst = df.rename(columns={0:'description',1:'value'})
df_col_rst

,description,value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [17]:
html_table = df_col_rst.to_html()
# html_table_2 = df_2.to_html()
# html_table_3 = df_3.to_html()
# html_table
# html_table_2
# html_table

In [18]:
clean_html_table = html_table.replace('\n', '')
# html_table_2.replace('\n', '')
# html_table_3.replace('\n', '')
# clean_html_table

In [19]:
# add table to mongodb as an object?
collection = db.partiii
html_table_obj = {
    'table': clean_html_table
}
collection.insert_one(html_table_obj)

In [20]:
df_col_rst.to_html('table.html')
# df_2.to_html('table.html')
# df_3.to_html('table.html')

In [21]:
# !open table.html

In [22]:
# play around with xpath
chromedriver_location = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars')


In [23]:
# wait for page to fully load
time.sleep(10)
# init list hemisphere_image_urls list
hemisphere_image_urls = []
for i in range(4):
    time.sleep(5)
    # xpath for thumbnail
    thumb_btn = f'//*[@id="product-section"]/div[2]/div[{i+1}]/a/img'
    # print(thumb_btn)
    # click thumbnail
    driver.find_element_by_xpath(thumb_btn).click()
    # wait for page to load
    time.sleep(5)
    # extract title
    title = driver.find_element_by_class_name('title')
    print(title.text)
    time.sleep(5)
    # xpath for full image
    sample_xpath = '//*[@id="wide-image"]/div/ul/li[1]/a'
    elem = driver.find_element_by_xpath(sample_xpath)
    # extract url of image
    link = elem.get_attribute('href')
    print(link)
    # create dictionary & append hemisphere_image_urls list
    url = {
        'title':title.text,'img_url':link
    }
    hemisphere_image_urls.append(url)
    # extra link to tif file
    # orig_xpath = '//*[@id="wide-image"]/div/ul/li[2]/a'
    # elem_two = driver.find_element_by_xpath(orig_xpath)
    # link_two = elem.get_attribute('href')
    # print(link_two)
    time.sleep(5)
    driver.back()
hemisphere_image_urls

Cerberus Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg
Schiaparelli Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg
Syrtis Major Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg
Valles Marineris Hemisphere Enhanced
http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg


[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'http://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]

In [24]:
time.sleep(10)
# add hemisphere_image_urls list to partiv collection
collection = db.partiv
collection.insert_many(hemisphere_image_urls)

In [41]:
# twitter stuff
weath_xpath = '//*[@id="react-root"]/div/div/div/main/div/div/div/div[1]/div/div/div/div/div[2]/section/div/div/div/div[1]/div/article/div/div[2]/div[2]/div[2]/span'



In [42]:
# play around with xpath
chromedriver_location = '/usr/local/bin/chromedriver'
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://twitter.com/marswxreport?lang=en')


In [43]:
time.sleep(5)
weath_elem = driver.find_element_by_xpath(weath_xpath)
weath_elem_txt = weath_elem.text
clean_w_elem = weath_elem_txt.replace('\n', '')
twitter_weather = {
    'weather':clean_w_elem
}

In [44]:
collection = db.partv
collection.insert_one(twitter_weather)